# top: TODO

<details>
<summary>
7.30
</summary>

1. `pro` -> lower(`Pro- Sports Giao Thuy JSC`)
2. `dap cau` -> lower(`Dap Cau - Yen Phong JSC`)
3. `branch of xxx` -> `xxx`
4. `tai` -> lower(`Tai - Yuan Garments`)
</details>

<details>
<summary>
7.31
</summary>

1. check `doubt==1` [x]
2. match `branch of...`
3. convert handcoded=1 for string "NA"
4. check round=0
</details>






In [282]:
import pandas as pd
import numpy as np
import json
import sqldf
# handcoded=pd.read_excel("../w8/BWID_factoryname handcoded.xlsx", sheet_name='data')

In [234]:
bw=pd.read_csv('bw_w9.csv')
ci=pd.read_csv('ci_w9.csv')
match3=['fuma costume', # this match3 is all results in match3, for filtered, see m3.txt
 'alim',
 'i and j hcm',
 'j and d vinako',
 'long thanh branch',
 'roo hsing co. nicaragua sa',
 'crystal martin',
 'lee and suy vina',
 'phuong nam garment trading import export jsc',
 'phu tai linh service and trading',
 'w and d apparel corp',
 'avery dennison group',
 'new century clothing industry llc',
 'eins',
 'united apparel cambodia branch 2 (formerly lotus textile garment cambodia  branch 2)',
 'dha',
 'yi da viet nam ltd',
 'j t h textile (cambodia) inc. (formerly united apparel (cambodia) 3, formerly lotus 3',
 'elite',
 'park corp ltd',
 'g h i garments cambodia',
 'gildan activewear rivas ii, sa',
 'zhong yin b textile',
 'seil',
 'tainan enterprises',
 'casual wear apparel llc',
 'hong kong winfair garment ltd',
 'emperor',
 'fashion garment 2',
 'opus one corp',
 'quint major industrial vietnam',
 'atto',
 'jaya perkasa textile pt',
 's w j garment',
 'southern garment manufacturing',
 'evergreen apparel',
 'yu fa garment industry',
 'my viet international export service trade',
 'e m b jeilkovi',
 'my hung garment and trading jsc',
 'a and j carter ltd',
 'premier global garment factory',
 't and k garment industry',
 'nesia pan pacific clothing pt',
 'yu da garment industry (cambodia)  (formerly maxpearl garment)',
 'indo jordan clothing company',
 'ty fashion (cambodia) plc. (no.6) (formerly t y fashion (cambodia)  (branch 2)',
 'nesia pan pacific knit pt',
 'm and j garment-printing-embroidery',
 'hai duong garment stock company no.2',
 'dress shirts ltd',
 'h and h textiles, sa',
 'youngone nam dinh',
 'smart shirts enterprise ltd',
 'casual wear apparel',
 'xuan hoa garment',
 'van thanh trading service',
 'h and l apparel corp',
 'g and g ii garments',
 'fashion garments 2',
 'international trading services ltd',
 'rina jaya garment pt',
 'ty fashion (cambodia) plc. (no.5) (formerly t y fashion (cambodia) )',
 'j t h textile inc',
 'd and blue',
 'pan pacific jakarta cabang semarang pt',
 'ty fashion (cambodia) plc. (formerly no.4 and no.5)',
 'leading star garment',
 'dong a textile garment jsc',
 'nam and co london',
 'classic fashion concept ltd',
 'g and g fashion',
 'tan uy dat',
 'laspo pt',
 'dae kwang apparel jsc']

bw['handcoded']=0
bw['round']=0
bw.loc[bw.cName==bw.RRCleaned,'round']=1
bw.loc[bw.cName.isin(match3),'round']=3

# function def.

In [355]:
def getFirst2(s):
    if len(s.split(" "))<2:
        return s
    
    op=" ".join(s.split(" ")[:2])
    if len(s.split(" "))>2:
        op=op+" "
    return op

def ciInfoGet(cName):
    global ci
    return ci[ci.cName==cName]

def select1(name):
    return bw[bw.cName==name].head(1)
    
def searchCI(kw,country=None):
    global ci
    if country==None:
        return sqldf.run(
        f'''
        select distinct company_name, sectors, headquarters_country, url 
        from ci 
        where company_name like '%{kw}%'
        group by company_name''')
    else:
        return sqldf.run(
            f'''
            select distinct company_name, sectors, headquarters_country, url 
            from ci 
            where company_name like '%{kw}%' and headquarters_country= '{country}'
            group by company_name''')

def match(bwName, ciName,round=0,doubt=0):
    global bw
    
    bw.loc[bw.cName==bwName, "round"]=round
    bw.loc[bw.cName==bwName, "CNRR"]=ci.loc[ci.cName==ciName,"company_name"].values[0]
    bw.loc[bw.cName==bwName, "repriskID"]=ci.loc[ci.cName==ciName,"repriskID"].values[0]
    bw.loc[bw.cName==bwName, "RRCleaned"]=ciName
    bw.loc[bw.cName==bwName, "RRSector"]=ci.loc[ci.cName==ciName,"sectors"].values[0]
    bw.loc[bw.cName==bwName, "doubt"]=doubt

# show all match3 corresponding results:

In [161]:
for i in match3:
    bwc=bw[bw.cName==i].Country.unique()[0]
    poss=ci[(ci.cName.str.contains(getFirst2(i))) & (ci.headquarters_country==bwc) ].cName.unique()
    if len(poss)==0:
        continue
    print("#",i," : ",poss)

# fuma costume  :  ['new fuma costume']
# i and j hcm  :  ['rai and rohl technologies inc'
 'thuong mai and khai thac khoang san duong hieu ctcp']
# long thanh branch  :  ['sonadezi long thanh shareholding co' 'long thanh plastic']
# lee and suy vina  :  ['vietnam lee and man paper manufacturing ltd']
# phuong nam garment trading import export jsc  :  ['phuong nam cultural jsc'
 'phuong nam education investment and development jsc']
# phu tai linh service and trading  :  ['phu tai jsc']
# elite  :  ['green elite group']
# gildan activewear rivas ii, sa  :  ['gildan activewear san marcos ii sa']
# zhong yin b textile  :  ['zhong yin cambodia b textile']
# jaya perkasa textile pt  :  ['jatim jaya perkasa pt' 'pakindo jaya perkasa pt' 'mitra jaya perkasa cv'
 'arumba jaya perkasa pt' 'nusa jaya perkasa pt'
 'rajawali jaya perkasa pt' 'solusi jaya perkasa pt'
 'munte waniq jaya perkasa pt']
# evergreen apparel  :  ['sd evergreen apparel']
# my hung garment and trading jsc  :  ['phu my hung

# match corresponding cols in from match3

In [235]:
with open('m3.txt') as m3:
    m3=m3.read().split("\n")
    m3.remove("")

# m3
for i in m3:
    i=i.replace("#","")
    # .replace("[","").replace("]","").replace("'","")
    doubt = 1 if i.startswith("*") else 0
    i=i.replace("*","")
    bwName=i.split(":")[0].strip()
    ciNames=eval(i.split(":")[1].strip()) #list type
    # print(bwName,ciNames,doubt)
    if len(ciNames)==1:
        ciNameCorr=ciNames[0]
        bw.loc[bw.cName==bwName, "CNRR"]=ci.loc[ci.cName==ciNameCorr,"company_name"].values[0]
        bw.loc[bw.cName==bwName, "repriskID"]=ci.loc[ci.cName==ciNameCorr,"repriskID"].values[0]
        bw.loc[bw.cName==bwName, "RRCleaned"]=ciNameCorr
        bw.loc[bw.cName==bwName, "RRSector"]=ci.loc[ci.cName==ciNameCorr,"sectors"].values[0]
        bw.loc[bw.cName==bwName, "doubt"]=doubt

In [237]:
# ci[ci.cName.str.contains("t and k ")]

---
# currently bw:

In [186]:
bw.loc[bw.cName=='pro','cName']='Pro- Sports Giao Thuy JSC'.lower()
bw.loc[bw.cName=='dap cau','cName']='Dap Cau - Yen Phong JSC'.lower()
bw.loc[bw.cName=='tai','cName']='Tai - Yuan Garments'.lower()

# `m2-1`:

In [218]:
with open('../w8/m2-1_new.txt') as m2_1:
    m2_1=[x for x in m2_1.read().replace("'","").replace("[",'').replace("]","").replace("\n","").split('$') if x!=""]

In [210]:
bwCName=list(bw.cName.unique())
ciCName=list(ci.cName.unique())

In [253]:
for i in m2_1:
    doubt=0
    bwName=i.split(":")[0].strip()
    ciName=i.split(":")[1].strip()
    if i.startswith("*"):
        doubt=1
        bwName=bwName[1:].strip()
    
    bw.loc[bw.cName==bwName, "round"]=2

flamingo fashions limited;flamingo fashions ltd

jinnat knitwears limited;jinnat knit wears ltd

debonair limited;debonair ltd


# `m2-n`

In [275]:
with open ("../w8/m2-n.txt") as m2n:
    m2n=[x for x in m2n.read().replace("'","").replace("[",'').replace("]","").replace("\n","").split('$') if x!="" and x.startswith("~")==False]

## test:
# for i in m2n:
#     if i.split(":")[1].strip() not in ciCName: #bwCName
#         print(i)

for i in m2n:
    doubt=0
    bwName=i.split(":")[0].strip()
    ciName=i.split(":")[1].strip()
    if i.startswith("*"):
        doubt=1
        bwName=bwName[1:].strip()
    bw.loc[bw.cName==bwName, "round"]=2
    bw.loc[bw.cName==bwName, "CNRR"]=ci.loc[ci.cName==ciName,"company_name"].values[0]
    bw.loc[bw.cName==bwName, "repriskID"]=ci.loc[ci.cName==ciName,"repriskID"].values[0]
    bw.loc[bw.cName==bwName, "RRCleaned"]=ciName
    bw.loc[bw.cName==bwName, "RRSector"]=ci.loc[ci.cName==ciName,"sectors"].values[0]
    bw.loc[bw.cName==bwName, "doubt"]=doubt

    # if bw.loc[bw.cName==bwName,'RRCleaned'].isna().any()==True:
    #     print(bwName)

In [276]:
bw[bw.cName=='sam kyung jaya busana pt']

,QuestID,FactoryBWID,FactoryName,Country,AssessmentDate,year,FNC,cName,CNRR,repriskID,RRCleaned,RRSector,doubt,handcoded,round
4770,17087,2071,PT Sam Kyung Jaya Busana,Indonesia,11/10/2016,2016,NaN,sam kyung jaya busana pt,PT Samkyung Jaya Busana,2635115.0,samkyung jaya busana pt,Retail,0,0,2
4771,23511,2071,PT Sam Kyung Jaya Busana,Indonesia,07/11/2017,2017,NaN,sam kyung jaya busana pt,PT Samkyung Jaya Busana,2635115.0,samkyung jaya busana pt,Retail,0,0,2
4772,30432,2071,PT Sam Kyung Jaya Busana,Indonesia,30/10/2018,2018,NaN,sam kyung jaya busana pt,PT Samkyung Jaya Busana,2635115.0,samkyung jaya busana pt,Retail,0,0,2
4773,36967,2071,PT Sam Kyung Jaya Busana,Indonesia,21/10/2019,2019,NaN,sam kyung jaya busana pt,PT Samkyung Jaya Busana,2635115.0,samkyung jaya busana pt,Retail,0,0,2


In [279]:
bw.drop('FNC',axis=1).to_csv('bw0730.csv', index=False)

---
# match handcoded:

In [372]:
handcoded=pd.read_excel("../w8/BWID_factoryname handcoded.xlsx", sheet_name='data',keep_default_na=False, na_values=[""])
handcodedIdx=handcoded[handcoded.RRCleaned.isna()==False].FactoryBWID.unique()

In [354]:
bw[bw.cName.str.contains('sarah')]
# match('sarah','sarah textile',round=2)

,QuestID,FactoryBWID,FactoryName,Country,AssessmentDate,year,FNC,cName,CNRR,repriskID,RRCleaned,RRSector,doubt,handcoded,round
193,11342,81,"Sarah Co., Ltd.",Vietnam,13/10/2015,2015,NaN,sarah,Sarah Textile Co Ltd (Sarah International; Con...,2233058.0,sarah textile,Personal and Household Goods,0,1,2
194,17022,81,"Sarah Co., Ltd.",Vietnam,18/10/2016,2016,NaN,sarah,Sarah Textile Co Ltd (Sarah International; Con...,2233058.0,sarah textile,Personal and Household Goods,0,1,2
195,24344,81,"Sarah Co., Ltd.",Vietnam,19/12/2017,2017,NaN,sarah,Sarah Textile Co Ltd (Sarah International; Con...,2233058.0,sarah textile,Personal and Household Goods,0,1,2
196,38490,81,"Sarah Co., Ltd.",Vietnam,11/02/2020,2020,NaN,sarah,Sarah Textile Co Ltd (Sarah International; Con...,2233058.0,sarah textile,Personal and Household Goods,0,1,2
197,42186,81,"Sarah Co., Ltd.",Vietnam,28/04/2021,2021,NaN,sarah,Sarah Textile Co Ltd (Sarah International; Con...,2233058.0,sarah textile,Personal and Household Goods,0,1,2


In [348]:
ci[ci.cName.str.contains("sarah")]

,repriskID,company_name,headquarters_country,headquarters_country_isocode,sectors,url,isins,primary_isin,no_reported_risk_exposure,cName
4855,2233058,Sarah Textile Co Ltd (Sarah International; Con...,Vietnam,VN,Personal and Household Goods,None,None,None,1,sarah textile


---
# check `doubt==1`

In [287]:
desiredSectors=['Chemicals', 'General Industrials', 'Industrial Engineering', 'Personal and Household Goods', 'Retail', 'Support Services (Industrial Goods and Services)', 'Unspecified']
doubted=['indo jordan clothing company', 'fine apparel ltd',
       'al hanan for cloth manufacturing est', 'al masera textile',
       'pan pacific jakarta cabang semarang pt',
       'international elegance garment manufacturing',
       'al kawthar garments industry and trade',
       'eclat highter international garment (cambodia)  (formerly highter)',
       'everay textile garment', 'ever-glory garment manufacturing',
       'e-z international garment factory', 'fuma costume',
       't and k garment industry', 'gildan activewear rivas ii, sa',
       'van thanh trading service', 'eminent garment ltd',
       'indo-chine vn', 'knit and knitwears ltd', 'creative wash ltd',
       'mascot knits ltd', 'al mustamerah for tex', 'sc enterprises pt',
       'thats it garments ltd', 'phu tai linh service and trading',
       'thats it sweater ltd', 'dress shirts ltd', 'gunung abadi pt',
       'bac giang lng garment corp', 'grupo alfatech,sa',
       'duc hanh garment jsc', 'long thanh branch', 'js vina',
       'jmtech busana global pt', 'sees vina']

In [328]:
select1('long thanh branch')

,QuestID,FactoryBWID,FactoryName,Country,AssessmentDate,year,FNC,cName,CNRR,repriskID,RRCleaned,RRSector,doubt,handcoded,round
6118,32710,2910,"Long Thanh Branch - Yupoong Viet Nam Co., Ltd.",Vietnam,05/03/2019,2019,NaN,long thanh branch,Long Thanh Plastic Co Ltd,62829.0,long thanh plastic,General Industrials | Personal and Household G...,1,0,3


In [330]:
searchCI('long thanh')

,company_name,sectors,headquarters_country,url
0,Long Thanh Plastic Co Ltd,General Industrials | Personal and Household G...,Vietnam,http://www.lothacolt.com
1,Sonadezi Long Thanh Shareholding Co,Construction and Materials,Vietnam,http://www.szl.com.vn


In [360]:
def removeBranch(str):
    if "branch of" not in str:
        return str
    else:
        return str.replace("branch of","").strip()

removeBranch('branch of thuan phuong embroideries garments')

'thuan phuong embroideries garments'

In [359]:
bw[bw.cName.str.contains('branch of')].cName.unique()

array(['branch of poong in vina', 'branch of continent packaging corp',
       'branch of truong vinh trading industrial garment',
       'branch of thuan phuong embroideries garments',
       'branch of garment 10 corp jsc',
       'branch of smart elegant international vietnam ltd. no.2',
       'branch of yupoong vietnam',
       'branch of ha hae vietnam corp in thai binh',
       'branch of hanoi textile and garment jsc', 'branch of leatherbank',
       'branch of dap cau garment corp jsc',
       'branch of viet tien garment corp'], dtype=object)

# export

In [361]:
bw.to_csv("F:/bw0731.csv", index=False)

In [380]:
ci[ci.repriskID==22694]

,repriskID,company_name,headquarters_country,headquarters_country_isocode,sectors,url,isins,primary_isin,no_reported_risk_exposure,cName
4970,22694,Musa Garments Factory,Jordan,JO,Personal and Household Goods,None,None,None,0,musa garments factory
